In [49]:
import cv2
import numpy as np

def rescale_image(image, target_width):
    height, width = image.shape[:2]
    scale_factor = target_width / width
    new_width = int(width * scale_factor)
    new_height = int(height * scale_factor)
    resized_image = cv2.resize(image, (new_width, new_height))
    return resized_image

def convert_hue(hue):
    return cv2.applyColorMap((hue*(255/179)).astype('uint8'), cv2.COLORMAP_HSV)

def count_circles(im):
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    # custom params
    circles = cv2.HoughCircles(im ,cv2.HOUGH_GRADIENT, 1, 50, param1=100,param2=20,minRadius=10, maxRadius=50)
    result = 0
    if circles is not None:
        circles = np.uint16(np.around(circles))
        result = len(circles[0,:])

        """
        for i in circles[0,:]:
            cv2.circle(im ,(i[0],i[1]),i[2],(0,255,0),2)
        """
            
    return result
    

def count_coins(img):
    img = rescale_image(img, 640)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    mask_yellow = cv2.inRange(img_hsv, (20, 100, 0), (50, 255, 255))
    mask_yellow = cv2.medianBlur(mask_yellow, 5)
    mask_blue = cv2.inRange(img_hsv, (90, 100, 100), (120, 255, 255))
    mask_blue = cv2.medianBlur(mask_blue, 5)

    im_yellow = cv2.bitwise_and(img, img, mask=mask_yellow)
    yellow = count_circles(im_yellow)
    
    im_blue = cv2.bitwise_and(img, img, mask=mask_blue)
    blue = count_circles(im_blue)

    #cv2.imshow('mask yellow', mask_yellow)
    #cv2.imshow('mask blue', mask_blue)
    cv2.putText(img, f'y: {yellow} b: {blue}', (50, 50), cv2.FONT_HERSHEY_PLAIN, 5, (0,0,0), 2)
    cv2.imshow('img', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return blue, yellow
    

In [50]:
for i in range(1, 11):
    img = cv2.imread(f"coin{i}.jpg")
    blue, yellow = count_coins(img)
    print(f'img {i} has {blue} coins and {yellow} coins')

img 1 has 8 coins and 5 coins
img 2 has 3 coins and 6 coins
img 3 has 4 coins and 2 coins
img 4 has 4 coins and 2 coins
img 5 has 7 coins and 1 coins
img 6 has 5 coins and 3 coins
img 7 has 3 coins and 4 coins
img 8 has 5 coins and 5 coins
img 9 has 6 coins and 2 coins
img 10 has 2 coins and 4 coins


In [3]:
def on_mouse_click (event, x, y, flags, frame):
    if event == cv2.EVENT_LBUTTONUP:
        print(frame[y, x].tolist())


In [40]:
img = cv2.imread("coin3.jpg")
img = rescale_image(img, 640)

# remove light?
"""
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
th = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
img = cv2.bitwise_and(img, img, mask=th)
"""

img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#kernel = np.ones((5, 5), np.uint8) 

mask_yellow = cv2.inRange(img_hsv, (20, 100, 0), (50, 255, 255))
mask_yellow = cv2.medianBlur(mask_yellow, 5)

mask_blue = cv2.inRange(img_hsv, (90, 100, 100), (120, 255, 255))
mask_blue = cv2.medianBlur(mask_blue, 5)
#mask_blue = cv2.erode(mask_blue, kernel)
#mask_blue = cv2.dilate(mask_blue, kernel)

"""
contours_yellow, hierarchy_yellow = cv2.findContours(mask_yellow, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
contours_blue, hierarchy_blue = cv2.findContours(mask_blue, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

yellow = len(contours_yellow)
blue = len(contours_blue)
"""

# if use hough circle, we just win :D
t = cv2.bitwise_and(img, img, mask=mask_blue)
t = cv2.cvtColor(t, cv2.COLOR_BGR2GRAY)
circles = cv2.HoughCircles(t ,cv2.HOUGH_GRADIENT, 1, 50, param1=100,param2=20,minRadius=10, maxRadius=50)
if circles is not None:
    circles = np.uint16(np.around(circles))
    print(len(circles[0,:]))
    for i in circles[0,:]:
        # draw the outer circle
        cv2.circle(img,(i[0],i[1]),i[2],(0,255,0),2)
        # draw the center of the circle
        #cv.circle(im,(i[0],i[1]),2,(0,0,255),3)

#cv2.imshow('img', img)
#cv2.imshow('t', t)

#print(blue)
#cv2.imshow('mask yellow', mask_yellow)
#cv2.drawContours(img_hsv, contours_blue, -1, (0,255,0), 3)
cv2.imshow('mask blue', mask_blue)

cv2.imshow('hue', img_hsv)
cv2.setMouseCallback('hue', on_mouse_click, img_hsv)
cv2.waitKey(0)
cv2.destroyAllWindows()

4
[30, 6, 91]
[19, 22, 93]
[91, 235, 141]
